# Лабораторная работа №5 по курсу "Технологии машинного обучения"
## Выполнила Попова Дарья, студентка группы РТ5-61Б

## Ансамбли моделей машинного обучения. Регрессия

Продолжим использовать датасет из ЛР №2-4 с данными американских граждан, в котором независимыми переменными являются:
* пол,
* возраст,
* индекс массы тела,
* число детей,
* является ли человек курильщиком (бинарный признак),
* регион проживания (территория США поделена на 4 части),

а целевым признаком - стоимость медицинской страховки (charges).

In [1]:
import pandas as pd
import numpy as np
insurance = pd.read_csv('C:\\Users\\Дасупс\\Downloads\\insurance.csv')

In [2]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Предобработка данных

In [3]:
# убедимся в том, что в данных нет пропусков
insurance.isnull().any()

age         False
sex         False
bmi         False
children    False
smoker      False
region      False
charges     False
dtype: bool

In [4]:
# закодируем One-Hot Encoding'ом категориальные признаки
insurance = pd.get_dummies(insurance)

In [5]:
# отмасштабируем числовые признаки со значениями возраста и индексом массы тела
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
insurance[['age']] = minmax_scaler.fit_transform(insurance[['age']])
insurance[['bmi']] = minmax_scaler.fit_transform(insurance[['bmi']])

In [7]:
insurance.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.021739,0.321227,0,16884.92400,1,0,0,1,0,0,0,1
1,0.000000,0.479150,1,1725.55230,0,1,1,0,0,0,1,0
2,0.217391,0.458434,3,4449.46200,0,1,1,0,0,0,1,0
3,0.326087,0.181464,0,21984.47061,0,1,1,0,0,1,0,0
4,0.304348,0.347592,0,3866.85520,0,1,1,0,0,1,0,0


In [11]:
len(insurance.columns)

12

## Разделение данных

In [13]:
X = insurance.drop(['charges'], axis=1)
y = insurance.charges

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Построение и обучение ансамблей 

## Случайный лес

Сначала обучим случайный лес с произвольными параметрами.

Концепция случайного леса состоит в том, что для каждой отдельной случайной выборки строится решающее дерево на основании случайного набора признаков. Объём этого набора можно регулировать гиперпараметром max_features.

Случайный лес ориентирован на борьбу с переобучением и, соответственно, нацелен (вместе с бэггингом) на уменьшение дисперсии. Случайный лес хорошо работает на данных модели, склонных к переобучению, в которых нет сложных зависимостей.

In [8]:
from sklearn.ensemble import RandomForestRegressor

Сначала обучим случайный лес на произвольных значениях гиперпараметров.

In [46]:
rand_forest = RandomForestRegressor(n_estimators=5, max_features=4, n_jobs=-1, random_state=42)

In [47]:
rand_forest.fit(X_train, y_train)

RandomForestRegressor(max_features=4, n_estimators=5, n_jobs=-1,
                      random_state=42)

In [48]:
rand_forest_predicted = rand_forest.predict(X_test)

In [60]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def print_metrics(y_true, y_predicted):
    print("{:<15} {:<15}".format('метрика','значение'))
    print()
    print("{:<15} {:<15}".format('r2_score', round(r2_score(y_true, y_predicted),3)))
    print("{:<15} {:<15}".format('MAE', round(mean_absolute_error(y_true, y_predicted),1)))
    print("{:<15} {:<15}".format('MSE', round(mean_squared_error(y_true, y_predicted),1)))

In [61]:
print_metrics(y_test, rand_forest_predicted)

метрика         значение       

r2_score        0.817          
MAE             3010.4         
MSE             27659481.0     


Теперь попробуем найти оптимальные гиперпараметры с помощью решётчатого поиска.

In [83]:
from sklearn.model_selection import GridSearchCV
params = {'n_estimators':range(1,30,2), 'max_features':range(1,6), 'max_depth':range(1,20,2)}

In [84]:
%%time
forest_grid = GridSearchCV(RandomForestRegressor(), params, n_jobs=-1)
forest_grid.fit(X_train, y_train)

Wall time: 1min


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(1, 20, 2),
                         'max_features': range(1, 6),
                         'n_estimators': range(1, 30, 2)})

In [85]:
forest_grid.best_params_

{'max_depth': 5, 'max_features': 5, 'n_estimators': 23}

In [86]:
best_forest_predicted = forest_grid.best_estimator_.predict(X_test)
print_metrics(y_test, best_forest_predicted)

метрика         значение       

r2_score        0.866          
MAE             2639.6         
MSE             20239163.4     


Как и можно было ожидать, метрики качества значительно улучшились.

! Задать вопрос Юрию Евгеньевичу!

В лекции было сказано, что в случае задачи регрессии для D фичей оптимальным является значение dl = D/3. 
Его же отсчитывать уже после всего препроцессинга данных и появление новых колонов за счёт OHE?

## Сверхслучайные деревья

В отличие от случайного леса, сверхслучайные деревья вносят ещё бОльшую долю случайности: и строчки, и фичи по-прежнему выдёргиваются случайным образом, но теперь мы и пороговые значения признаков в разделяющих узлах будем выбирать случайно, а не оптимально.

In [90]:
%%time

from sklearn.ensemble import ExtraTreesRegressor
params = {'n_estimators':range(1,50,5), 'max_features':range(1,6), 'max_depth':range(1,20,2)}
extra_grid = GridSearchCV(ExtraTreesRegressor(), params, n_jobs=-1)
extra_grid.fit(X_train, y_train)

Wall time: 45.6 s


GridSearchCV(estimator=ExtraTreesRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(1, 20, 2),
                         'max_features': range(1, 6),
                         'n_estimators': range(1, 50, 5)})

In [91]:
extra_grid.best_params_

{'max_depth': 9, 'max_features': 5, 'n_estimators': 26}

Обратим внимание, что модель сверхслучайных деревьев подобрала другие оптимальные значения гиперпараметров.

In [92]:
extra_trees_pred = extra_grid.best_estimator_.predict(X_test)
print_metrics(y_test, extra_trees_pred)

метрика         значение       

r2_score        0.85           
MAE             2744.1         
MSE             22635366.2     


Результат получился даже похуже.

## Градиентный бустинг

Градиентный бустинг борется скорее со смещением и строит модель на основании более сложных зависимостей.

Идея заключается в следующем:
Градиентный бустинг обучает первую модель на целевом признаке.
Вторую модель - на разнице между предсказаниями первой модели и целевого признака.
Третью - на разнице между предсказаниями второй и целевым признаком и так далее.

Каждая модель пытается скомпенсировать ошибку, каждый раз уменьшая её степень.

Проверим, как справится встроенный в sklearn градиентный бустинг.

In [77]:
from sklearn.ensemble import GradientBoostingRegressor

In [79]:
# default n_estimators=100

grad_boost = GradientBoostingRegressor(random_state=42)
grad_boost.fit(X_train, y_train)
boost_predicted = grad_boost.predict(X_test)

print_metrics(y_test, boost_predicted)

метрика         значение       

r2_score        0.861          
MAE             2543.0         
MSE             20937924.0     


In [82]:
# попробуем намеренно уменьшить число моделей

grad_boost = GradientBoostingRegressor(n_estimators=10, random_state=42)
grad_boost.fit(X_train, y_train)
boost_predicted = grad_boost.predict(X_test)

print_metrics(y_test, boost_predicted)

метрика         значение       

r2_score        0.763          
MAE             4502.0         
MSE             35786877.3     


При уменьшении числа моделей результат ухудшается.